## LangChain Expression Language(LCEL)

https://python.langchain.com/v0.1/docs/expression_language/why/

### 기본 구조: 프롬프트 + 모델 + 출력 파서


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv


# API KEY 정보로드
load_dotenv()


True

In [2]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

os.environ['LANGCHAIN_PROJECT'] = 'LCEL'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[LANGCHAIN_PROJECT]
LCEL


In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging


# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL


### 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
  - `template`: 템플릿 문자열. 문자열 내에서 중괄호 `{}`는 변수를 나타냄
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의함

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트

In [4]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [5]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [6]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [7]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

### Chain 생성

#### LCEL(LangChain Expression Language)


```
chain = prompt | model | output_parser
```

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달


In [9]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe47efbbfa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe47efd2140>, root_client=<openai.OpenAI object at 0x7fe47efba080>, root_async_client=<openai.AsyncOpenAI object at 0x7fe47efd00a0>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### invoke() 호출

- python 딕셔너리 형태(키: 값)로 입력값을 전달
- invoke() 함수 호출 시, 입력값을 전달

In [10]:
# input 딕셔너리에 주제 설정
input = {"topic": "인공지능의 학습 방법"}

In [11]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

AIMessage(content='인공지능의 학습 방법은 크게 지도학습, 비지도학습, 강화학습으로 나눌 수 있습니다.\n\n1. 지도학습: 지도학습은 입력 데이터와 그에 대한 정답이 주어졌을 때 인공지능 모델이 주어진 입력 데이터로부터 정답을 예측하도록 학습하는 방법입니다. 예를 들어, 고양이와 강아지 사진이 주어졌을 때, 모델은 각 사진이 고양이인지 강아지인지를 분류하는 작업을 학습할 수 있습니다.\n\n2. 비지도학습: 비지도학습은 정답이 주어지지 않은 데이터를 학습하는 방법으로, 데이터 간의 패턴이나 구조를 발견하도록 모델을 학습시킵니다. 클러스터링, 차원 축소, 생성 모델 등이 비지도학습의 예시입니다.\n\n3. 강화학습: 강화학습은 환경과 상호작용하며 보상을 최대화하는 방향으로 학습하는 방법입니다. 모델은 행동을 선택하고, 그 결과로 얻는 보상을 통해 스스로 학습하고 개선해 나갑니다. 게임이나 자율주행 자동차 등의 분야에서 주로 사용됩니다.\n\n이러한 학습 방법들을 조합하여 다양한 인공지능 모델을 학습시키고 다양한 문제를 해결할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 440, 'prompt_tokens': 30, 'total_tokens': 470, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0112829c-e546-45ef-b335-4aa3094b095c-0', usage_metadata

In [12]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능은 데이터를 입력받아 학습을 진행하는데, 그 학습 방법에는 크게 지도학습, 비지도학습, 강화학습이 있습니다.

1. 지도학습: 지도학습은 레이블이 달려 있는 데이터를 활용하여 모델을 학습시키는 방법입니다. 예를 들어, 사진과 그에 해당하는 레이블(고양이, 개 등)을 학습시켜 사진 속 동물을 분류할 수 있습니다.

2. 비지도학습: 비지도학습은 레이블 없이 데이터만을 이용하여 모델을 학습시키는 방법입니다. 이 방법은 데이터 간의 패턴이나 특징을 찾아내는 데 사용됩니다.

3. 강화학습: 강화학습은 에이전트가 환경과 상호작용하며 보상을 최대화하는 방향으로 학습하는 방법입니다. 예를 들어, 게임에서 승리할 때마다 보상을 받아 승리를 최대화하는 전략을 학습할 수 있습니다.

이러한 학습 방법을 통해 인공지능은 데이터를 분석하고 패턴을 학습하여 다양한 작업을 수행할 수 있습니다.

### 출력파서(Output Parser)


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [14]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe47efbbfa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe47efd2140>, root_client=<openai.OpenAI object at 0x7fe47efba080>, root_async_client=<openai.AsyncOpenAI object at 0x7fe47efd00a0>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [15]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능의 학습 원리"}
chain.invoke(input)

'인공지능은 기계가 데이터를 학습하여 패턴을 파악하고 문제를 해결하는 능력을 갖게 하는 기술입니다. \n\n인공지능은 먼저 대량의 데이터를 입력받고, 이를 분석하여 패턴을 발견합니다. 이후에는 발견한 패턴을 기반으로 문제를 해결하는 방법을 찾아내고, 그 결과를 토대로 스스로 학습하며 성능을 개선해나갑니다.\n\n간단히 말해, 인공지능은 데이터를 받아들이고 분석하여 문제를 해결하는 방법을 학습하는 과정을 반복함으로써 지능을 향상시키는 것입니다.'

In [16]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능의 학습 원리는 주로 기계학습이라는 분야에서 다뤄집니다. 기계학습은 컴퓨터 프로그램이 데이터를 통해 패턴을 학습하고 결정을 내리는 방법을 의미합니다. 

기계학습은 크게 지도학습, 비지도학습, 강화학습으로 나눌 수 있습니다. 

- 지도학습: 입력된 데이터와 그에 대한 정답(레이블)을 이용하여 모델을 학습시키는 방법입니다. 모델은 데이터의 패턴을 파악하고 정답을 예측할 수 있도록 학습됩니다.

- 비지도학습: 입력된 데이터에 대한 정답이 주어지지 않고 모델이 데이터의 패턴을 스스로 찾아내는 방법입니다. 이를 통해 데이터 간의 관계나 구조를 파악하거나 군집화하는 등의 작업을 수행할 수 있습니다.

- 강화학습: 환경과 상호작용하면서 보상을 최대화하는 방향으로 학습하는 방법입니다. 모델은 특정 행동의 결과로 받는 보상을 최대화하기 위해 시행착오를 통해 학습하며, 점진적으로 더 나은 행동을 선택하도록 학습됩니다.

이러한 기계학습 방법을 통해 인공지능은 데이터를 분석하고 패턴을 파악하여 문제를 해결하거나 예측하는 능력을 향상시킬 수 있습니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 가능
- `model_name` 역시 변경하여 테스트가 가능

In [18]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [19]:
# 체인을 구성
chain = prompt | model | output_parser

In [20]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
  **Waiter:** Good evening! Welcome to our restaurant. How many people are in your party?  
  **You:** Good evening! It's just me.  
  **Waiter:** Great! Here’s the menu. Can I start you off with something to drink?  
  **You:** Yes, I’d like a glass of water, please.  
  **Waiter:** Sure! Are you ready to order your meal?  
  **You:** Yes, I’ll have the grilled chicken with a side salad, please.  
  **Waiter:** Perfect choice! Would you like any dressing with your salad?  
  **You:** Yes, please. I’d like ranch dressing.  
  **Waiter:** Excellent! I’ll get that order in for you.  
  **You:** Thank you!  

- 한글 해석:  
  **웨이터:** 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?  
  **당신:** 좋은 저녁입니다! 저 혼자입니다.  
  **웨이터:** 좋습니다! 메뉴입니다. 음료는 무엇을 드릴까요?  
  **당신:** 네, 물 한 잔 주세요.  
  **웨이터:** 알겠습니다! 식사 주문하실 준비가 되셨나요?  
  **당신:** 네, 구운 치킨과 사이드 샐러드를 주문할게요.  
  **웨이터:** 좋은 선택입니다! 샐러드에 어떤 드레싱을 원하시나요?  
  **당신:** 네, 랜치 드레싱으로 주세요.  
  **웨이터:** 훌륭합니다! 주문해드리겠습니다.  
  **당신:** 감사합니다!  


In [21]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Waiter: Good evening! Welcome to our restaurant. How many people are in your party?
  - You: Good evening! It's just me.
  - Waiter: Great! Here’s the menu. Can I get you something to drink while you look it over?
  - You: Yes, I’d like a glass of water, please.
  - Waiter: Sure! Are you ready to order, or do you need a few more minutes?
  - You: I think I’m ready. I would like the grilled chicken salad, please.
  - Waiter: Excellent choice! Would you like any dressing with that?
  - You: Yes, please. I’d like ranch dressing.
  - Waiter: Perfect! I’ll have that right out for you. 

- 한글 해석:
  - 웨이터: 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 몇 분이신가요?
  - 당신: 좋은 저녁입니다! 저 혼자입니다.
  - 웨이터: 좋습니다! 여기 메뉴입니다. 메뉴를 살펴보시는 동안 음료를 드릴까요?
  - 당신: 네, 물 한 잔 주세요.
  - 웨이터: 알겠습니다! 주문하실 준비가 되셨나요, 아니면 몇 분 더 필요하신가요?
  - 당신: 준비가 된 것 같아요. 구운 치킨 샐러드 하나 주세요.
  - 웨이터: 훌륭한 선택입니다! 드레싱은 어떤 걸 원하시나요?
  - 당신: 네, 랜치 드레싱 주세요.
  - 웨이터: 완벽합니다! 바로 준비해 드리겠습니다.

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

### [실습] 템플릿을 변경하여 나만의 여행 가이드 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 여행 가이드
2. 3일간 가성비 여행 계획을 세워주는 챗봇 생성
3. `{question}` 에는 여행갈 나라와 도시를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [26]:
tour_guide_template = """
당신은 10년차 여행 가이드입니다. 주어진 질문에 맞는 3일간 가성비 여행 계획을 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.
각 계획마다 이동이 필요할 경우 도보 및 추천 이동수단을 적어주고 마지막은 숙소도 추천해 주세요.
#여행갈 나라와 도시:
{question}
#FORMAT:
-여행갈 나라와 도시의 정보
-추천 이동수단
-1일차 계획 
-2일차 계획
-3일차 계획
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(tour_guide_template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

# 체인을 구성
chain = prompt | model | output_parser

In [27]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "일본 도쿄 여행"})
# 스트리밍 출력
stream_response(answer)

# 일본 도쿄 여행

## 추천 이동수단
도쿄에서는 대중교통이 매우 발달해 있습니다. 지하철 및 JR(일본철도)를 이용하는 것이 가장 편리하며, 도보로 이동할 수 있는 거리도 많습니다. Suica 카드 또는 Pasmo 카드를 구입하여 편리하게 대중교통을 이용하세요.

---

### 1일차 계획
- **오전**: 
  - 아사쿠사 방문
    - 센소지(浅草寺) 사원 탐방
    - 나카미세 거리에서 기념품 쇼핑 및 간식 즐기기
- **점심**: 
  - 아사쿠사 근처의 전통적인 일본식 소바 또는 우동집에서 점심
- **오후**: 
  - 우에노 공원 및 우에노 동물원 탐방
    - 도보로 이동 (아사쿠사에서 우에노까지 약 20분)
- **저녁**: 
  - 아키하바라 방문
    - 전자제품 및 애니메이션 상품 쇼핑
- **숙소 추천**: 
  - 아사쿠사 근처의 '숙소 아사쿠사' (가성비 좋은 호스텔)

---

### 2일차 계획
- **오전**: 
  - 시부야 방문
    - 시부야 스크램블 교차로 체험
    - 하치코 동상 보기
- **점심**: 
  - 시부야 근처의 유명한 카레집에서 점심
- **오후**: 
  - 하라주쿠 탐방
    - 다케시타 거리에서 쇼핑 및 거리 음식 즐기기
- **저녁**: 
  - 오모테산도 거리 탐방
    - 카페에서 휴식 후 저녁
- **숙소 추천**: 
  - 시부야 근처의 '시부야 그랜드 호텔' (편리한 위치)

---

### 3일차 계획
- **오전**: 
  - 도쿄 타워 방문
    - 전망대에서 도쿄 전경 감상
- **점심**: 
  - 도쿄 타워 근처의 일식 레스토랑에서 점심
- **오후**: 
  - 롯폰기 힐즈 방문
    - 미술관 탐방 및 쇼핑
    - 도보로 이동 (도쿄 타워에서 롯폰기까지 약 20분)
- **저녁**: 
  - 긴자 탐방
    - 쇼핑 후 유명한 스시집에서 저녁
- **숙소 추천**: 
  - 긴자 근처의 '긴자 텐노우' (가성비 좋은 비즈니스 호텔)

---

이

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청

# 스트리밍 출력


### [실습] 템플릿을 변경하여 나만의 요리사 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 셰프
2. 냉장고 속 재료(여러 재료도 가능)를 입력으로 받아 요리명과 레시피 출력
3. `{food}` 에는 여행갈 냉장고 속 재료를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [ ]:
# 코드 작성

### [실습] 템플릿을 변경하여 나만의 헬스 트레이너 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 헬스 트레이너
2. 운동하고 싶은 신체 부위를 입력하면 운동 루틴을 출력 
3. `{today}` 에는 운동하고 싶은 신체 부위를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [ ]:
# 코드 작성